# 💻 Modeling microbiota-wide metabolism with MICOM


# 📝 Setup

MICOM installation

In [ ]:
!git clone https://github.com/SchuShoe/Micro_biome materials
%cd materials

Cloning into 'materials'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 122 (delta 40), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (122/122), 50.28 MiB | 20.60 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/materials


In [ ]:
# Merged data: https://github.com/SchuShoe/File_merge

## Basic Installation

Installing MICOM is straight-forward in Python. OSQP itself can be installed right along with it.

In [ ]:
!pip install -q osqp cobra micom

print("Done! 🎉 ")

     |████████████████████████████████| 2.6 MB 5.0 MB/s 
     |████████████████████████████████| 828 kB 41.9 MB/s 
     |████████████████████████████████| 84 kB 2.1 MB/s 
     |████████████████████████████████| 10.9 MB 28.0 MB/s 
     |████████████████████████████████| 7.3 MB 39.1 MB/s 
     |████████████████████████████████| 44 kB 2.5 MB/s 
     |████████████████████████████████| 229 kB 56.5 MB/s 
     |████████████████████████████████| 109 kB 53.7 MB/s 
     |████████████████████████████████| 147 kB 53.5 MB/s 
     |████████████████████████████████| 2.3 MB 39.6 MB/s 
     |████████████████████████████████| 68 kB 5.6 MB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 51 kB 5.3 MB/s 
     |████████████████████████████████| 546 kB 53.8 MB/s 
     |████████████████████████████████| 37.5 MB 80.5 MB/s 
Done! 🎉 


## Enable QIIME 2 interactions

Finally we will need to install packages to read the "biom" format which is a file format QIIME 2 uses to save tables. This is only necessary if you want to read QIIME 2 FeatureTable artifacts.

In [ ]:
!pip install -q numpy Cython
!pip install -q biom-format

print("Done! 🎉 ")

     |████████████████████████████████| 11.7 MB 5.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Done! 🎉 



## Importing data from QIIME 2

Obtain genome-scale models from the [AGORA database](https://pubmed.ncbi.nlm.nih.gov/27893703/), which is a curated database of more than 800 bacterial strains that commonly live in the human gut. In particular, we will use a version of this database summarized on the genus rank which can be downloaded from [MICOM data repository](https://doi.org/10.5281/zenodo.3755182) which contains a whole lot of prebuilt databases.



In [ ]:
!wget -O agora103_genus.qza https://zenodo.org/record/3755182/files/agora103_genus.qza?download=1

--2022-04-19 10:05:47--  https://zenodo.org/record/3755182/files/agora103_genus.qza?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21080080 (20M) [application/octet-stream]
Saving to: ‘agora103_genus.qza’

agora103_genus.qza  100%[===================>]  20.10M  12.6MB/s    in 1.6s    

2022-04-19 10:07:31 (12.6 MB/s) - ‘agora103_genus.qza’ saved [21080080/21080080]



We will remove all taxa that make up less than 2.5% of the community to keep the models small and speed up this tutorial.The data from the prior analysis can be found in the `treasure_chest` folder, so we can use those files.

In [ ]:
from micom.taxonomy import qiime_to_micom

tax = qiime_to_micom(
    "treasure_chest/dada2/table.qza", 
    "treasure_chest/taxa.qza", 
    collapse_on="genus"
)

Notice the `collapse_on` argument.
`collapse_on=["kingdom", "phylum", "class", "order", "family", "genus"]`. 

Note: the more ranks you use for matching the more likely are you to run into those issues. However, the more taxonomic ranks you use to match the more confident you can be that your observed taxon really is the same taxon as the one in the model database.

We can also look at the generated MICOM taxonomy.

In [ ]:
tax

,sample_id,abundance,genus,id,relative
0,ERR1883195,27876.0,Bacteroides,Bacteroides,0.510306
1,ERR1883207,27998.0,Bacteroides,Bacteroides,0.469458
2,ERR1883210,44054.0,Bacteroides,Bacteroides,0.655644
3,ERR1883212,16062.0,Bacteroides,Bacteroides,0.279811
4,ERR1883225,5514.0,Bacteroides,Bacteroides,0.630604
...,...,...,...,...,...
494,ERR1883320,2.0,Barnesiella,Barnesiella,0.000037
495,ERR1883331,9.0,Barnesiella,Barnesiella,0.000142
496,ERR1883212,2.0,WAL_1855D,WAL_1855D,0.000035
497,ERR1883212,4.0,Finegoldia,Finegoldia,0.000070


One helpful thing to do is to merge in our metadata, so we'll have it at hand for the following steps.

In [ ]:
import pandas as pd

metadata = pd.read_table("metadata.tsv").rename(columns={"id": "sample_id"})
tax = pd.merge(tax, metadata, on="sample_id")
tax

,sample_id,abundance,genus,id,relative,disease_stat,description
0,ERR1883195,27876.0,Bacteroides,Bacteroides,0.510306,healthy,Donor1
1,ERR1883195,4749.0,Faecalibacterium,Faecalibacterium,0.086937,healthy,Donor1
2,ERR1883195,1768.0,Coprococcus,Coprococcus,0.032366,healthy,Donor1
3,ERR1883195,1780.0,Roseburia,Roseburia,0.032585,healthy,Donor1
4,ERR1883195,4095.0,Blautia,Blautia,0.074964,healthy,Donor1
...,...,...,...,...,...,...,...
481,ERR1883331,8.0,rc4-4,rc4_4,0.000126,healthy,Donor13
482,ERR1883331,10.0,Butyricimonas,Butyricimonas,0.000158,healthy,Donor13
483,ERR1883331,2.0,Pediococcus,Pediococcus,0.000032,healthy,Donor13
484,ERR1883331,3.0,Corynebacterium,Corynebacterium,0.000047,healthy,Donor13


With the taxonomic metadata, we can finally build our community-level models.

## 1 Building community models

With the data we have now, building our models is pretty easy. We just pass our taxonomy table and model database to MICOM. We will have to specify where to write the models. We will also run that in parallel over two threads. It should take around 5 minutes to finish.

In [ ]:
from micom.workflows import build
from micom import Community
import pandas as pd

manifest = build(tax, "agora103_genus.qza", "models", solver="osqp", 
                 cutoff=2.5e-2, threads=2)


Output()

In lower-biomass 16S amplicon sequencing samples from stool, many reads can match to food components or to host mitochondria and these hits probably do not contribute much to bacterial community metabolism. These hits will be excluded from MICOM. 

Let's also take a look what we got back from the `build` process.

In [ ]:
manifest

,sample_id,disease_stat,description,file,found_taxa,total_taxa,found_fraction,found_abundance_fraction
0,ERR1883195,healthy,Donor1,ERR1883195.pickle,10.0,10.0,1.000000,0.908249
1,ERR1883207,healthy,Donor2,ERR1883207.pickle,8.0,8.0,1.000000,0.850601
2,ERR1883210,healthy,Donor3,ERR1883210.pickle,6.0,6.0,1.000000,0.847675
3,ERR1883212,healthy,Donor4,ERR1883212.pickle,9.0,10.0,0.900000,0.874048
4,ERR1883225,healthy,Donor5,ERR1883225.pickle,5.0,5.0,1.000000,0.882091
5,ERR1883247,healthy,Donor6,ERR1883247.pickle,9.0,11.0,0.818182,0.840901
6,ERR1883261,healthy,Donor7,ERR1883261.pickle,7.0,8.0,0.875000,0.747772
7,ERR1883273,healthy,Donor8,ERR1883273.pickle,8.0,8.0,1.000000,0.891107
8,ERR1883285,healthy,Donor9,ERR1883285.pickle,7.0,9.0,0.777778,0.772151
9,ERR1883293,healthy,Donor10,ERR1883293.pickle,2.0,3.0,0.666667,0.666667


This will tell you many taxa were found in the database and what fraction of the total abundance was represented by the database. Looks okay here. 

So we now have our community models and can leverage MICOM fully by simulating community growth.

## 2 Growth simulation for community models

With our community models built, we can start to simulate growth with the cooperative tradeoff algorithm. 

In [ ]:
!wget https://github.com/Gibbons-Lab/isb_course_2021/raw/main/setup_qiime2
%run setup_qiime2

--2022-04-19 10:20:40--  https://github.com/Gibbons-Lab/isb_course_2021/raw/main/setup_qiime2
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Gibbons-Lab/isb_course_2021/main/setup_qiime2 [following]
--2022-04-19 10:20:41--  https://raw.githubusercontent.com/Gibbons-Lab/isb_course_2021/main/setup_qiime2
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4974 (4.9K) [text/plain]
Saving to: ‘setup_qiime2.1’

setup_qiime2.1      100%[===================>]   4.86K  --.-KB/s    in 0s      

2022-04-19 10:20:41 (36.9 MB/s) - ‘setup_qiime2.1’ saved [4974/4974]



[10:20:44] 🐍 Downloading miniconda...                                        ]8;id=288758;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=753461;file:///content/materials/setup_qiime2#56\56]8;;\

[10:20:45] 🐍 Done.                                                           ]8;id=20294;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=82935;file:///content/materials/setup_qiime2#62\62]8;;\

           🐍 Installing miniconda...                                         ]8;id=34501;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=482270;file:///content/materials/setup_qiime2#56\56]8;;\

[10:21:04] 🐍 Installed miniconda to `/usr/local`.                            ]8;id=367852;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=918991;file:///content/materials/setup_qiime2#62\62]8;;\

...

[10:29:02] 🌲 Done.                                                           ]8;id=398;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=669692;file:///content/materials/setup_qiime2#62\62]8;;\

           📊 Checking that Qiime 2 command line works...                     ]8;id=720225;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=640552;file:///content/materials/setup_qiime2#56\56]8;;\

[10:29:06] 📊 Qiime 2 command line looks good 🎉                              ]8;id=95594;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=490474;file:///content/materials/setup_qiime2#62\62]8;;\

           🧹 Cleaned up unneeded files.                                      ]8;id=792025;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=163183;file:///content/materials/setup_qiime2#51\51]8;;\

           Everything is A-OK. You can start using Qiime 2 now 👍            ]8;id=875325;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=30892;file:///content/materials/setup_qiime2#159\159]8;;\

Load a medium of choice into Micom:
1. vegeterian: /content/materials/media/vegeterian.alo.qza
2. meat: /content/materials/media/meat.alo.qza
3. gluten free: /content/materials/media/gluten_free.alo.qza
4. high fiber: /content/materials/media/high_fiber.alo.qza
5. high fats low carbs: /content/materials/media/high_fat_low_carb_agora.qza

In [ ]:
from micom.qiime_formats import load_qiime_medium

medium = load_qiime_medium("/content/materials/media/vegeterian.alo.qza")
medium.flux *= 10
medium

,reaction,metabolite,global_id,flux
reaction,,,,
EX_4hbz_m,EX_4hbz_m,4hbz_m,EX_4hbz(e),0.000794
EX_ala_L_m,EX_ala_L_m,ala_L_m,EX_ala_L(e),107.055729
EX_arab_D_m,EX_arab_D_m,arab_D_m,EX_arab_D(e),0.007503
EX_arg_L_m,EX_arg_L_m,arg_L_m,EX_arg_L(e),74.424967
EX_asp_L_m,EX_asp_L_m,asp_L_m,EX_asp_L(e),146.457270
...,...,...,...,...
EX_nadp_m,EX_nadp_m,nadp_m,EX_nadp(e),0.000310
EX_urea_m,EX_urea_m,urea_m,EX_urea(e),8.461282
EX_h2_m,EX_h2_m,h2_m,EX_h2(e),5.807145


Many dietary components get absorbed in the small intestine. This medium was created by taking dietary components and depleting all nutrients absorbed in the small intestine by a factor of 10 (indicated by the dilution column). 

## Simulating growth

In [ ]:
from micom.workflows import grow

growth_results = grow(manifest, "models", medium, tradeoff=0.5, threads=2)

Output()

/usr/local/lib/python3.7/dist-packages/micom/workflows/grow.py:199: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  exchanges["taxon"] = exchanges.index


 `grow` returns a tuple of 3 data sets:

1. The predicted growth rate for all taxa in all samples
2. The import and export fluxes for each taxon and the external environment
3. Annotations for the fluxes mapping to other databases



In [ ]:
growth_results.growth_rates.head()

,abundance,growth_rate,reactions,metabolites,taxon,tradeoff,sample_id
compartments,,,,,,,
Alistipes,0.036623,0.595981,2650,1613,Alistipes,0.5,ERR1883195
Bacteroides,0.561858,9.126655,3307,1887,Bacteroides,0.5,ERR1883195
Blautia,0.082537,1.338268,3108,1818,Blautia,0.5,ERR1883195
Coprococcus,0.035635,0.576243,2188,1591,Coprococcus,0.5,ERR1883195
Dorea,0.029528,0.478622,2368,1614,Dorea,0.5,ERR1883195


## 📊 Visualizations

Use the standard visualizations included in MICOM and visualize the results on the browser.

###😀 **Remember to download the results!!!** 

The first things we might want to look at are the growth rates for each taxon.

In [ ]:
from micom.viz import *

viz = plot_growth(growth_results)



## Growth niches

Two really important questions are 'what dietary nutrients are consumed by the microbiota and what metabolites do the microbiota produce?' We provided nutrients in our medium, but we don't actually know yet what was eaten by the microbiota. Let's check that out using the `plot_exchanges_per_sample` function.

In [ ]:
plot_exchanges_per_sample(growth_results)

We can have a look at the results after downloading `materials/sample_exchanges_[DATE].html`. It would be even better if we could visualize which taxa compete for similar resources. We can create a niche plot by using `plot_exchanges_per_taxon`.

In [ ]:
plot_exchanges_per_taxon(growth_results, perplexity=4)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,



This function projects the full set of import or export fluxes onto a two dimensional plane, and arranges taxa so that more similar flux patterns lie close together. Thus, taxa closer to one another compete for a more similar set of resources (and/or produce a more similar set of metabolites). The center of the plot signifies a more competitive nutrient space, whereas clusters on the outskirts denote more isolated niches.

## 3 Effects of diets on individual fluxes # 

We will choose five samples for individual analysis and comparison.

First, recall what samples we had.

In [ ]:
manifest

,sample_id,disease_stat,description,file,found_taxa,total_taxa,found_fraction,found_abundance_fraction
0,ERR1883195,healthy,Donor1,ERR1883195.pickle,10.0,10.0,1.000000,0.908249
1,ERR1883207,healthy,Donor2,ERR1883207.pickle,8.0,8.0,1.000000,0.850601
2,ERR1883210,healthy,Donor3,ERR1883210.pickle,6.0,6.0,1.000000,0.847675
3,ERR1883212,healthy,Donor4,ERR1883212.pickle,9.0,10.0,0.900000,0.874048
4,ERR1883225,healthy,Donor5,ERR1883225.pickle,5.0,5.0,1.000000,0.882091
5,ERR1883247,healthy,Donor6,ERR1883247.pickle,9.0,11.0,0.818182,0.840901
6,ERR1883261,healthy,Donor7,ERR1883261.pickle,7.0,8.0,0.875000,0.747772
7,ERR1883273,healthy,Donor8,ERR1883273.pickle,8.0,8.0,1.000000,0.891107
8,ERR1883285,healthy,Donor9,ERR1883285.pickle,7.0,9.0,0.777778,0.772151
9,ERR1883293,healthy,Donor10,ERR1883293.pickle,2.0,3.0,0.666667,0.666667


We will **pick a diet of choice**:
1. vegeterian.alo.qza
2. meat.alo.qza
3. gluten_free.alo.qza
4. high_fiber.alo.qza
5. high_fat_low_carb_agora.qza

Then, we will use the **five subjects** from below and **run them individually with a selected diet**.
1. ERR1883195.pickle
2. ERR1883207.pickle	
3. ERR1883210.pickle	
4. ERR1883212.pickle
5. ERR1883261.pickle	


In [ ]:
from micom import load_pickle
from micom.qiime_formats import load_qiime_medium

medium = load_qiime_medium("/content/materials/media/vegeterian.alo.qza")
medium.flux *= 10
medium.index = medium.reaction

com = load_pickle('models/ERR1883331.pickle')
com.medium = medium.flux
com

[16:07:12] WARNING  I could not find the following exchanges in your model:  ]8;id=358923;file:///usr/local/lib/python3.7/dist-packages/micom/community.py\community.py]8;;\:]8;id=104043;file:///usr/local/lib/python3.7/dist-packages/micom/community.py#670\670]8;;\
                    EX_nadp_m, EX_amet_m, EX_tet_m, EX_chsterol_m,                           
                    EX_5mthf_m, EX_cmp_m, EX_datp_m, EX_gtp_m, EX_thf_m                      

Name,ERR1883331
Memory address,0x07f2c2e2b5f90
Number of metabolites,14059
Number of reactions,22275
Number of groups,0
Objective expression,1.0*community_objective
Compartments,"c__Bacteroides, e__Bacteroides, m, e__Faecalibacterium, c__Faecalibacterium, c__Blautia, e__Blautia, c__Alistipes, e__Alistipes, c__Dorea, e__Dorea, c__Ruminococcus, e__Ruminococcus, c__Akkermansia, e__Akkermansia, c__Parabacteroides, e__Parabacteroides"


## Performing knockout ⛔ on individual bacteria :

In [ ]:
from micom import Community, data

ko = com.knockout_taxa(fraction=0.5)
ko

## Individual growth and community growth rates

Run the cooperative tradeoff for a single model and follow that up with FBA to get all fluxes in the system.

###🌝 **Remember to copy the results to a spread sheet!!!**

The obtained values would be used for data analysis and comparison between individuals using the same diet.

In [ ]:
sol = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=False)
sol

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Akkermansia,0.038070,1.915581,2274,1386
Alistipes,0.051183,2.577222,2650,1613
Bacteroides,0.631472,31.639744,3307,1887
Blautia,0.109497,5.462193,3108,1818
Dorea,0.038767,1.939289,2368,1614
Faecalibacterium,0.044155,2.217327,1986,1472
Parabacteroides,0.036513,1.832526,2870,1747
Ruminococcus,0.050343,2.522697,2983,1793
medium,NaN,NaN,729,729


## Check the bacteria that contribute most to SCFA production
Check the fluxes for the three most abundant short-chain fatty acids in human gut: acetate, butyrate, and proprionate.

###😏 **Remember to copy the results to a spread sheet!!!**

In [ ]:
# Acetate
sol.fluxes["EX_ac(e)"]* com.abundances

In [ ]:
# Butyrate
sol.fluxes["EX_but(e)"]* com.abundances

In [ ]:
# Proprionate
sol.fluxes["EX_ppa(e)"]* com.abundances

After obtaining the fluxe values we are gonna sum them up to obtain the **net flux**, which can then be compared across the individuals.

In [ ]:
#Check for the most active fluxes
highest = sol.fluxes.abs().mean(axis=0).sort_values(ascending=False)[0:5000]
highest = highest.to_frame(name="flux").reset_index()
highest 